# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load the csv file from Part I
file = "output_data/weather_per_city.csv"
weather_data = pd.read_csv(file)
weather_data


,Unnamed: 0,City,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Lat,Lng
0,0,Karratha,101.84,28,47,22.95,AU,1638768846,-20.7377,116.8463
1,1,San Cristobal,71.78,80,100,2.04,VE,1638768746,7.7669,-72.2250
2,2,Province of Mantua,40.44,91,91,3.00,IT,1638768846,45.1667,10.7833
3,3,Maroantsetra,80.24,74,68,5.17,MG,1638768847,-15.4333,49.7333
4,4,Sinop Province,41.43,70,17,12.12,TR,1638768847,41.7711,34.8709
...,...,...,...,...,...,...,...,...,...,...
585,585,Pihuamo,66.78,80,100,2.98,MX,1638769101,19.2499,-103.3828
586,586,Gīdolē,64.65,41,9,4.81,ET,1638769101,5.6500,37.3667
587,587,Krishnapur,75.02,94,90,11.50,IN,1638769102,22.6703,88.2694
588,588,Rafaela,70.05,88,2,6.64,AR,1638769102,-31.2503,-61.4867


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = weather_data[["Lat", "Lng"]]
weight = weather_data["Humidity"]


In [4]:

fig_layout= {"width": "400px", "height": "300px", "border": "1px solid black", "padding": "1px",
                 "margin": "0 auto 0 auto"}

fig = gmaps.figure(layout=fig_layout, center=(10,0), zoom_level=2)

max_humidity = weather_data["Humidity"].max()
heat_layer = gmaps.heatmap_layer(locations, weights = weight, dissipating=False, max_intensity=80, point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# max temp 70-80 F, windspeed less than 10, cloudiness 0
nice_weather = weather_data.loc[(weather_data['Max Temp'] < 80) & (weather_data['Max Temp'] > 70) & (weather_data['Wind Speed'] < 10) & (weather_data['Cloudiness'] == 0)].copy()
nice_weather = nice_weather.dropna(how="any")
nice_weather = nice_weather.reset_index()
nice_weather



,index,Unnamed: 0,City,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Lat,Lng
0,69,69,Pochutla,74.82,75,0,3.87,MX,1638768872,15.7432,-96.4661
1,228,228,Sur,77.38,53,0,2.98,OM,1638768939,22.5667,59.5289
2,321,321,Bajil,79.23,58,0,2.95,YE,1638768980,15.0584,43.2851
3,396,396,Nizwá,72.88,41,0,4.18,OM,1638769015,22.9333,57.5333
4,564,564,Hukuntsi,74.19,58,0,8.84,BW,1638769091,-24.0000,21.7500
5,573,573,Buin,74.73,65,0,1.12,CL,1638769095,-33.7333,-70.7500


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = nice_weather
hotel_df["Hotel Name"]=""
hotel_df

,index,Unnamed: 0,City,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Lat,Lng,Hotel Name
0,69,69,Pochutla,74.82,75,0,3.87,MX,1638768872,15.7432,-96.4661,
1,228,228,Sur,77.38,53,0,2.98,OM,1638768939,22.5667,59.5289,
2,321,321,Bajil,79.23,58,0,2.95,YE,1638768980,15.0584,43.2851,
3,396,396,Nizwá,72.88,41,0,4.18,OM,1638769015,22.9333,57.5333,
4,564,564,Hukuntsi,74.19,58,0,8.84,BW,1638769091,-24.0000,21.7500,
5,573,573,Buin,74.73,65,0,1.12,CL,1638769095,-33.7333,-70.7500,


In [7]:
#Hit Google Places API for city coordinates
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
search = "Hotel"
radius = 5000
stype = "Lodging"
        
#make empty list of hotels
hotel_list = []

x = 0

#iterate through hotels in cities that fit the parameters and store hotel names
for city in hotel_df["City"]:
    target_coordinates = str(hotel_df["Lat"][x]) + ', ' + str(hotel_df["Lng"][x])
    x = x + 1
    
    params={"location":target_coordinates, "keyword":search, "radius":radius, "type":stype,
           "key":g_key}
    
    try:
        response = requests.get(base_url, params=params).json()
        hotel_name = response['results'][0]['name']
        hotel_list.append(hotel_name)
    
    except IndexError:
        hotel_list.append(np.nan)
    
hotel_list

['Hotel San Pedro',
 'Sur Plaza Hotel',
 'Tourism Ambassadors Hotel',
 'Falaj Daris Hotel',
 'Entabeni Hotel',
 'Hostal Condell']

In [8]:
#add list of hotels to hotel name column in df
hotel_df["Hotel Name"] = hotel_list
hotel_df

,index,Unnamed: 0,City,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Lat,Lng,Hotel Name
0,69,69,Pochutla,74.82,75,0,3.87,MX,1638768872,15.7432,-96.4661,Hotel San Pedro
1,228,228,Sur,77.38,53,0,2.98,OM,1638768939,22.5667,59.5289,Sur Plaza Hotel
2,321,321,Bajil,79.23,58,0,2.95,YE,1638768980,15.0584,43.2851,Tourism Ambassadors Hotel
3,396,396,Nizwá,72.88,41,0,4.18,OM,1638769015,22.9333,57.5333,Falaj Daris Hotel
4,564,564,Hukuntsi,74.19,58,0,8.84,BW,1638769091,-24.0000,21.7500,Entabeni Hotel
5,573,573,Buin,74.73,65,0,1.12,CL,1638769095,-33.7333,-70.7500,Hostal Condell


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)

# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…